In [ ]:
!pip install eager optimum[onnxruntime] timm onnxruntime-gpu

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 6.1 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.4/429.4 kB 

In [5]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=7b7b7cad875c1aaf8e4994dbb369146802c3d49f0dc2de59ef694d99f5f7bc9e
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built gputil


In [6]:
import os
import time

import psutil
import GPUtil
import kagglehub
import numpy as np
from tqdm import tqdm

import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import torch
from torch import nn, optim
from torch.amp import autocast
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.utils import prune

from sklearn.metrics import precision_score, recall_score, f1_score

In [7]:
def print_memory_usage(label=""):
    """Выводит использование памяти в мегабайтах"""
    if label:
        print(f"\n--- Memory Usage ({label}) ---")
    else:
        print("\n--- Memory Usage ---")
        
    # CPU RAM в MB
    process = psutil.Process(os.getpid())
    ram_used = process.memory_info().rss / (1024 ** 2)
    print(f"CPU RAM used: {ram_used:.2f} MB")
    
    # GPU VRAM в MB
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        vram_used = gpu.memoryUsed
        vram_total = gpu.memoryTotal
        print(f"GPU {gpu.id} VRAM: {vram_used:.2f} MB / {vram_total:.2f} MB")


def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    return (param_size + buffer_size) / 1024**2


def calculate_metrics(model, device, data_loader):
    model.eval()
    all_preds = []
    all_targets = []

    model.to(device)
    with torch.no_grad():
        for images, targets in data_loader:
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            all_targets.extend(targets.numpy())
    
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    
    return precision, recall, f1


def load_imagenet_mini(dataset_path, model, split):
    config = resolve_data_config({}, model=model)
    transform = create_transform(**config)

    print(transform)

    if split == 'train':
        transform = transforms.Compose([
            transforms.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.2,
            ),
            transforms.GaussianBlur(
                kernel_size=3,
            ),
            transform,
        ])

    dataset = datasets.ImageFolder(
        root=os.path.join(dataset_path, split),
        transform=transform
    )
    
    return dataset


def benchmark_model(model, device, input_tensor, num_runs=10, warmup=3, use_amp=False):
    model = model.to(device)
    input_tensor = input_tensor.to(device)
    
    # Warmup
    print(f"\n🔥 Warming up ({warmup} runs) on {device}...")
    for _ in range(warmup):
        with torch.no_grad():
            if use_amp and device.type == 'cuda':
                with autocast(device_type='cuda', dtype=torch.float16):
                    _ = model(input_tensor)
            else:
                _ = model(input_tensor)
    
    # Benchmark
    print(f"🚀 Benchmarking ({num_runs} runs) on {device}...")
    start_time = time.time()
    
    for _ in range(num_runs):
        with torch.no_grad():
            if use_amp and device.type == 'cuda':
                with autocast(device_type='cuda', dtype=torch.float16):
                    _ = model(input_tensor)
            else:
                _ = model(input_tensor)
    
    total_time = (time.time() - start_time) * 1000
    avg_time = total_time / num_runs
    print(f"✅ Average inference: {avg_time:.2f} ms")
    print(f"📊 Total time: {total_time:.2f} ms | FPS: {1000/(avg_time + 1e-9):.1f}")
    
    return avg_time


def main(model, dataset_path=None):
    device_cpu = torch.device('cpu')
    device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print("\n🔍 Initial memory state:")
    print_memory_usage("Before loading model")
    model.eval()
    print(f"📏 Model size: {get_model_size(model):.2f} MB")

    print("\n🔍 Initial memory state:")
    print_memory_usage("Model loaded")
    
    # Бенчмарки
    input_tensor = torch.randn(1, 3, 224, 224)
    
    print("\n🧪 Benchmarking on CPU:")
    cpu_time = benchmark_model(model, device_cpu, input_tensor)
    print_memory_usage("After CPU benchmark")
    
    if torch.cuda.is_available():
        print("\n🎮 Benchmarking on GPU:")
        gpu_time = benchmark_model(model, device_gpu, input_tensor)
        print_memory_usage("After GPU test")
        
        print("\n⚡ Benchmarking with AMP:")
        gpu_amp_time = benchmark_model(model, device_gpu, input_tensor, use_amp=True)
        print_memory_usage("After AMP test")
        
        print("\n📈 Results Summary:")
        print(f"| Device | Inference Time (ms) | Speedup vs CPU |")
        print("|--------|---------------------|----------------|")
        print(f"| CPU    | {cpu_time:19.2f} | {'—':^15} |")
        print(f"| GPU    | {gpu_time:19.2f} | {cpu_time/gpu_time:^15.1f}x |")
        print(f"| AMP    | {gpu_amp_time:19.2f} | {cpu_time/gpu_amp_time:^15.1f}x |")
    else:
        print("\n❌ CUDA not available")
        print(f"⏱️ CPU inference time: {cpu_time:.2f} ms")
        if dataset_path:
            print("\n🎯 Quality Metrics (CPU):")
            print(f"Precision: {precision_cpu:.4f}")
            print(f"Recall:    {recall_cpu:.a4f}")
            print(f"F1-Score:  {f1_cpu:.4f}")

    # Загрузка и расчет метрик качества
    if dataset_path:
        print("\n📊 Loading ImageNetMini dataset...")
        val_dataset = load_imagenet_mini(dataset_path, model, 'val')
        data_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True, drop_last=False)
        
        # print("\n🧮 Calculating metrics on CPU:")
        # precision_cpu, recall_cpu, f1_cpu = calculate_metrics(model, device_cpu, data_loader)
        
        if torch.cuda.is_available():
            print("\n🧮 Calculating metrics on GPU:")
            precision_gpu, recall_gpu, f1_gpu = calculate_metrics(model, device_gpu, data_loader)

            print("\n🎯 Quality Metrics Summary:")
            print("| Device | Precision | Recall  | F1-Score |")
            print("|--------|-----------|---------|----------|")
            # print(f"| CPU    | {precision_cpu:.4f}  | {recall_cpu:.4f} | {f1_cpu:.4f}  |")
            print(f"| GPU    | {precision_gpu:.4f}  | {recall_gpu:.4f} | {f1_gpu:.4f}  |")

In [8]:
path = kagglehub.dataset_download("ifigotin/imagenetmini-1000")
path += "/imagenet-mini"

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imagenetmini-1000/imagenet-mini


In [9]:
base_model = timm.create_model('efficientvit_b3.r256_in1k', pretrained=True)

model.safetensors:   0%|          | 0.00/195M [00:00<?, ?B/s]

In [10]:
# main(base_model, path)

## Optimum optimization

In [11]:
def print_memory_usage(label=""):
    """Выводит использование памяти в мегабайтах"""
    if label:
        print(f"\n--- Memory Usage ({label}) ---")
    else:
        print("\n--- Memory Usage ---")
        
    # CPU RAM в MB
    process = psutil.Process(os.getpid())
    ram_used = process.memory_info().rss / (1024 ** 2)
    print(f"CPU RAM used: {ram_used:.2f} MB")
    
    # GPU VRAM в MB
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        vram_used = gpu.memoryUsed
        vram_total = gpu.memoryTotal
        print(f"GPU {gpu.id} VRAM: {vram_used:.2f} MB / {vram_total:.2f} MB")


def calculate_metrics(model, device, data_loader):
    all_preds = []
    all_targets = []

    model.to(device)
    with torch.no_grad():
        for images, targets in data_loader:
            images = images.to(device)
            outputs = model(images).logits
            preds = outputs.argmax(dim=1).cpu().numpy()
            
            all_preds.extend(preds)
            all_targets.extend(targets.numpy())
    
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')
    
    return precision, recall, f1


def load_imagenet_mini(dataset_path, model, split):
    config = resolve_data_config({}, model=base_model)
    transform = create_transform(**config)

    print(transform)

    dataset = datasets.ImageFolder(
        root=os.path.join(dataset_path, split),
        transform=transform
    )
    
    return dataset


def benchmark_model(model, device, input_tensor, num_runs=10, warmup=3, use_amp=False):
    model = model.to(device)
    input_tensor = input_tensor.to(device)
    
    # Warmup
    print(f"\n🔥 Warming up ({warmup} runs) on {device}...")
    for _ in range(warmup):
        with torch.no_grad():
            if use_amp and device.type == 'cuda':
                with autocast(device_type='cuda', dtype=torch.float16):
                    _ = model(input_tensor)
            else:
                _ = model(input_tensor)
    
    # Benchmark
    print(f"🚀 Benchmarking ({num_runs} runs) on {device}...")
    start_time = time.time()
    
    for _ in range(num_runs):
        with torch.no_grad():
            if use_amp and device.type == 'cuda':
                with autocast(device_type='cuda', dtype=torch.float16):
                    _ = model(input_tensor)
            else:
                _ = model(input_tensor)
    
    total_time = (time.time() - start_time) * 1000
    avg_time = total_time / num_runs
    print(f"✅ Average inference: {avg_time:.2f} ms")
    print(f"📊 Total time: {total_time:.2f} ms | FPS: {1000/(avg_time + 1e-9):.1f}")
    
    return avg_time


def main(model, dataset_path=None):
    device_cpu = torch.device('cpu')
    device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print("\n🔍 Initial memory state:")
    print_memory_usage("Before loading model")

    print("\n🔍 Initial memory state:")
    print_memory_usage("Model loaded")
    
    # Бенчмарки
    input_tensor = torch.randn(1, 3, 256, 256)
    
    print("\n🧪 Benchmarking on CPU:")
    cpu_time = benchmark_model(model, device_cpu, input_tensor)
    print_memory_usage("After CPU benchmark")
    
    if torch.cuda.is_available():
        print("\n🎮 Benchmarking on GPU:")
        gpu_time = benchmark_model(model, device_gpu, input_tensor)
        print_memory_usage("After GPU test")
        
        print("\n⚡ Benchmarking with AMP:")
        gpu_amp_time = benchmark_model(model, device_gpu, input_tensor, use_amp=True)
        print_memory_usage("After AMP test")
        
        print("\n📈 Results Summary:")
        print(f"| Device | Inference Time (ms) | Speedup vs CPU |")
        print("|--------|---------------------|----------------|")
        print(f"| CPU    | {cpu_time:19.2f} | {'—':^15} |")
        print(f"| GPU    | {gpu_time:19.2f} | {cpu_time/gpu_time:^15.1f}x |")
        print(f"| AMP    | {gpu_amp_time:19.2f} | {cpu_time/gpu_amp_time:^15.1f}x |")
    else:
        print("\n❌ CUDA not available")
        print(f"⏱️ CPU inference time: {cpu_time:.2f} ms")
        if dataset_path:
            print("\n🎯 Quality Metrics (CPU):")
            print(f"Precision: {precision_cpu:.4f}")
            print(f"Recall:    {recall_cpu:.a4f}")
            print(f"F1-Score:  {f1_cpu:.4f}")

    # Загрузка и расчет метрик качества
    if dataset_path:
        print("\n📊 Loading ImageNetMini dataset...")
        val_dataset = load_imagenet_mini(dataset_path, model, 'val')
        data_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True, drop_last=False)
        
        # print("\n🧮 Calculating metrics on CPU:")
        # precision_cpu, recall_cpu, f1_cpu = calculate_metrics(model, device_cpu, data_loader)
        
        if torch.cuda.is_available():
            print("\n🧮 Calculating metrics on GPU:")
            precision_gpu, recall_gpu, f1_gpu = calculate_metrics(model, device_gpu, data_loader)

            print("\n🎯 Quality Metrics Summary:")
            print("| Device | Precision | Recall  | F1-Score |")
            print("|--------|-----------|---------|----------|")
            # print(f"| CPU    | {precision_cpu:.4f}  | {recall_cpu:.4f} | {f1_cpu:.4f}  |")
            print(f"| GPU    | {precision_gpu:.4f}  | {recall_gpu:.4f} | {f1_gpu:.4f}  |")

In [12]:
from optimum.onnxruntime import ORTModelForImageClassification
from transformers import AutoFeatureExtractor
from pathlib import Path

2025-05-16 10:18:37.886700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747390718.054451      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747390718.100785      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
model_id="timm/efficientvit_b3.r256_in1k"
onnx_path = Path("onnx")

model = ORTModelForImageClassification.from_pretrained(model_id, export=True)
model.save_pretrained(onnx_path)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

You are using a model of type timm_wrapper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
/usr/local/lib/python3.11/dist-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:657: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/usr/local/lib/python3.11/dist-packages/torch/onnx/utils.py:1127: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >

In [14]:
!rm onnx/model.onnx

In [16]:
import torch

base_model.eval()

with torch.no_grad():
    torch.onnx.export(
        base_model.to('cuda'),
        torch.rand(1, 3, 256, 256).to('cuda'),
        'onnx/model.onnx',
        opset_version=15,
        input_names=['pixel_values'],
        output_names=['logits'],
        dynamic_axes={'pixel_values': {0: 'batch_size'}, 'logits': {0: 'batch_size'}},
    )

In [17]:
model_id="timm/efficientvit_b3.r256_in1k"
onnx_path = Path("onnx")

model = ORTModelForImageClassification.from_pretrained(onnx_path, file_name="model.onnx")

You are using a model of type timm_wrapper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


In [18]:
# main(model, path)

## Quantization with Optimum

'/head/classifier/classifier.4/Gemm'

In [27]:
import onnx

onnx_model_path = "onnx/model.onnx"
onnx_model = onnx.load(onnx_model_path)

exclude_nodes = []
for node in onnx_model.graph.node:
    if 'stages' in node.name:
        name_parts = node.name.split('/')

        stage_val = int(name_parts[2].split('.')[1])

        if stage_val >= 2:
            exclude_nodes.append(node.name)

print(exclude_nodes)

['/stages/stages.2/blocks/blocks.0/main/inverted_conv/conv/Conv', '/stages/stages.2/blocks/blocks.0/main/inverted_conv/act/HardSwish', '/stages/stages.2/blocks/blocks.0/main/depth_conv/conv/Conv', '/stages/stages.2/blocks/blocks.0/main/depth_conv/act/HardSwish', '/stages/stages.2/blocks/blocks.0/main/point_conv/conv/Conv', '/stages/stages.2/blocks/blocks.1/context_module/main/Shape', '/stages/stages.2/blocks/blocks.1/context_module/main/Constant', '/stages/stages.2/blocks/blocks.1/context_module/main/Gather', '/stages/stages.2/blocks/blocks.1/context_module/main/Shape_1', '/stages/stages.2/blocks/blocks.1/context_module/main/Constant_1', '/stages/stages.2/blocks/blocks.1/context_module/main/Gather_1', '/stages/stages.2/blocks/blocks.1/context_module/main/Shape_2', '/stages/stages.2/blocks/blocks.1/context_module/main/Constant_2', '/stages/stages.2/blocks/blocks.1/context_module/main/Gather_2', '/stages/stages.2/blocks/blocks.1/context_module/main/qkv/conv/Conv', '/stages/stages.2/block

In [28]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from onnxruntime.quantization import QuantFormat, QuantizationMode

# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(model)
qconfig = AutoQuantizationConfig.avx512_vnni(
    is_static=True,
    per_channel=True,
    nodes_to_exclude=exclude_nodes,
)

In [29]:
train_dataset = load_imagenet_mini(path, base_model, 'train')

Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(256, 256))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)


In [30]:
from torch.utils.data import random_split
from datasets import Dataset

calibration_size = 200
calibration_dataset, _ = random_split(train_dataset, [calibration_size, len(train_dataset) - calibration_size])
images_list = [image for image, _ in calibration_dataset]
optimum_dataset = Dataset.from_dict({
    "pixel_values": images_list,
})

In [31]:
from optimum.onnxruntime.configuration import AutoCalibrationConfig

calibration_config = AutoCalibrationConfig.percentiles(optimum_dataset, percentile=99.99)

# Perform the calibration step: computes the activations quantization ranges
batch_size = 10
shards = len(optimum_dataset) // batch_size

for i in range(shards):
    print(f'Processing {i + 1} shard')
    shard = optimum_dataset.shard(shards, i)
    quantizer.partial_fit(
        dataset=shard,
        calibration_config=calibration_config,
        operators_to_quantize=qconfig.operators_to_quantize,
        batch_size=batch_size,
        use_external_data_format=False,
    )
ranges = quantizer.compute_ranges()

# remove temp augmented model again
os.remove("augmented_model.onnx")

Processing 1 shard
Processing 2 shard
Processing 3 shard
Processing 4 shard
Processing 5 shard
Processing 6 shard
Processing 7 shard
Processing 8 shard
Processing 9 shard
Processing 10 shard
Processing 11 shard
Processing 12 shard
Processing 13 shard
Processing 14 shard
Processing 15 shard
Processing 16 shard
Processing 17 shard
Processing 18 shard
Processing 19 shard
Processing 20 shard
Finding optimal threshold for each tensor using 'percentile' algorithm ...
Number of tensors : 560
Number of histogram bins : 2048
Percentile : (0.010000000000005116,99.99)


In [32]:
from optimum.onnxruntime.preprocessors import QuantizationPreprocessor
from optimum.onnxruntime.preprocessors.passes import (
    ExcludeGeLUNodes,
    ExcludeLayerNormNodes,
    ExcludeNodeAfter,
    ExcludeNodeFollowedBy,
)


def create_quantization_preprocessor():
    # Create a quantization preprocessor to determine the nodes to exclude
    quantization_preprocessor = QuantizationPreprocessor()

    # Exclude the nodes constituting LayerNorm
    quantization_preprocessor.register_pass(ExcludeLayerNormNodes())
    # Exclude the nodes constituting GELU
    quantization_preprocessor.register_pass(ExcludeGeLUNodes())
    # Exclude the residual connection Add nodes
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Add", "Add"))
    # Exclude the Add nodes following the Gather operator
    quantization_preprocessor.register_pass(ExcludeNodeAfter("Gather", "Add"))
    # Exclude the Add nodes followed by the Softmax operator
    quantization_preprocessor.register_pass(ExcludeNodeFollowedBy("Add", "Softmax"))

    return quantization_preprocessor


# create processor
quantization_preprocessor = create_quantization_preprocessor()

# Quantize the same way we did for dynamic quantization!
quantizer.quantize(
    save_dir=onnx_path,
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
    preprocessor=quantization_preprocessor,
)

PosixPath('onnx')

In [33]:
import os

# get model file size
size = os.path.getsize(onnx_path / "model.onnx")/(1024*1024)
quantized_model = os.path.getsize(onnx_path / "model_quantized.onnx")/(1024*1024)

print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")

Model file size: 185.87 MB
Quantized Model file size: 156.25 MB


In [34]:
model = ORTModelForImageClassification.from_pretrained(onnx_path, file_name="model_quantized.onnx")

You are using a model of type timm_wrapper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
Too many ONNX model files were found in onnx/model_quantized.onnx ,onnx/model.onnx. specify which one to load by using the `file_name` and/or the `subfolder` arguments. Loading the file model_quantized.onnx in the subfolder onnx.


In [35]:
main(model, path)


🔍 Initial memory state:

--- Memory Usage (Before loading model) ---
CPU RAM used: 7355.32 MB
GPU 0 VRAM: 617.00 MB / 16384.00 MB

🔍 Initial memory state:

--- Memory Usage (Model loaded) ---
CPU RAM used: 7355.32 MB
GPU 0 VRAM: 617.00 MB / 16384.00 MB

🧪 Benchmarking on CPU:

🔥 Warming up (3 runs) on cpu...
🚀 Benchmarking (10 runs) on cpu...
✅ Average inference: 93.84 ms
📊 Total time: 938.36 ms | FPS: 10.7

--- Memory Usage (After CPU benchmark) ---
CPU RAM used: 7391.95 MB
GPU 0 VRAM: 617.00 MB / 16384.00 MB

🎮 Benchmarking on GPU:


2025-05-16 10:34:06.669503944 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 35 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-05-16 10:34:06.678475128 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-16 10:34:06.678492079 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.



🔥 Warming up (3 runs) on cuda...
🚀 Benchmarking (10 runs) on cuda...
✅ Average inference: 24.26 ms
📊 Total time: 242.58 ms | FPS: 41.2

--- Memory Usage (After GPU test) ---
CPU RAM used: 7293.81 MB
GPU 0 VRAM: 891.00 MB / 16384.00 MB

⚡ Benchmarking with AMP:


2025-05-16 10:34:07.553071982 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 35 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-05-16 10:34:07.563191374 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-16 10:34:07.563210146 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.



🔥 Warming up (3 runs) on cuda...
🚀 Benchmarking (10 runs) on cuda...
✅ Average inference: 23.93 ms
📊 Total time: 239.33 ms | FPS: 41.8

--- Memory Usage (After AMP test) ---
CPU RAM used: 7293.93 MB
GPU 0 VRAM: 891.00 MB / 16384.00 MB

📈 Results Summary:
| Device | Inference Time (ms) | Speedup vs CPU |
|--------|---------------------|----------------|
| CPU    |               93.84 |        —        |
| GPU    |               24.26 |       3.9      x |
| AMP    |               23.93 |       3.9      x |

📊 Loading ImageNetMini dataset...
Compose(
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(256, 256))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

🧮 Calculating metrics on GPU:


2025-05-16 10:34:17.202086752 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 35 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-05-16 10:34:17.211890919 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-16 10:34:17.211909716 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.



🎯 Quality Metrics Summary:
| Device | Precision | Recall  | F1-Score |
|--------|-----------|---------|----------|
| GPU    | 0.8060  | 0.7791 | 0.7670  |


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
